# Import

In [ ]:
!pip install pymysql
!pip install requests

import pymysql #Mysql
import pandas as pd #Dataframe Operation
import numpy as np #calculation
import matplotlib.pyplot as plt #ploting
from datetime import date #date
import datetime
import seaborn as sns  #ploting
from datetime import datetime
from itertools import chain
import requests
import json
from decimal import Decimal

from datetime import datetime,timedelta
e='RDS link'
u='user name'
p='password'
db='daatabase LINK'
connection=pymysql.connect(host=e,user=u,passwd=p,db=db)

In [ ]:
per=input("Enter the Conv % : ")
per=Decimal(per)

Enter the Conv % : 0.5


In [ ]:
per=Decimal(per)

In [ ]:
cursors3=connection.cursor()
cursors3.execute("""
select
BuddyData.Zone,
sename As service_area_name,
ABS(sum(if(BuddyData.status = 1,1,0))) as FreeBuddies,
ABS(sum(if(BuddyData.status = 2,1,0)))as Offlinebuddy,
ABS(sum(if(BuddyData.status = 3,1,0))) as OnJob
from
(
	select buddy.status,service_area.name as sename,bsa.service_area_id as 'Zone'
	from buddy
	left join buddy_service_areas bsa
    on buddy.id = bsa.buddy_id
    left join user
    on user.id = buddy.user_id
    left join service_area
    on bsa.service_area_id = service_area.id

    where user.is_active=1 and user.role_id=5
)as BuddyData where zone is  not null
group by 1;

""")
rows3=cursors3.fetchall()
data3=pd.DataFrame(rows3)
arra=[]
for col in range(0,len(data3.columns)):
  arra.append(cursors3.description[col][0])

data3.columns=arra
data3.dropna(inplace=True)
Area=data3[data3['FreeBuddies']>0]
Area['Invite']=Area['FreeBuddies']*100/per
Area.reset_index(inplace=True)
Area.reset_index(inplace=True)
Area['Invite']=Area['Invite'].astype('float')
Area

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,level_0,index,Zone,service_area_name,FreeBuddies,Offlinebuddy,OnJob,Invite
0,0,6,10058,Gulshan -1,2,16,0,400.0
1,1,7,10059,Gulshan 2,7,28,0,1400.0
2,2,8,10060,Gulshan -3,7,25,0,1400.0
3,3,9,10061,Gulshan 4,6,23,0,1200.0
4,4,10,10023,PECHS - Naheed Supermarket,10,35,3,2000.0
5,5,11,10026,Shell Embassy Inn (Model),10,36,3,2000.0
6,6,12,10053,PECHS - AKU & KDA,10,32,3,2000.0
7,7,13,10057,PECHS - Shahrae Qaideen,1,0,0,200.0
8,8,14,26,Soldier Bazar - Pick & Save Supermarket,7,54,10,1400.0
9,9,15,10017,Saddar Area,1,36,4,200.0


In [ ]:
print("Total User will get Promo : ",Area['Invite'].sum())

Total User will get Promo :  16200.0


# Fields

In [ ]:
Code = 'HAPPYSNACKING' #@param {type:"string"}
Title = 'Extra snacking TODAY!' #@param {type:"string"}
Description = 'Use code HAPPYSNACKING and get 30% off on all snacks' #@param {type:"string"}
Next_Hours = '2' #@param {type:"string"}
Discount_Percentage = '30' #@param {type:"string"}
Minimum_Order_Amount = '200' #@param {type:"string"}
max_Discount_Per_Order = '150' #@param {type:"string"}
Max_Promo_Code_Usage_Count = '1000' #@param {type:"string"}
Max_Promo_Code_Usage_Count_per_user = '1' #@param {type:"string"}
title = 'Extra snacking TODAY!' #@param {type:"string"}
body = 'Use code HAPPYSNACKING and get 30% off on all snacks' #@param {type:"string"}
name = 'SNACKS' #@param {type:"string"}



# Trigger

In [ ]:
def Run():
  cursors2=connection.cursor()
  cursors2.execute("""
  select *,CONVERT_TZ(orders.accepted_at, '+00:00', '+5:00') As accepted_at_P from orders
  inner join address
  on orders.address_id=address.id
  inner join service_area
  on address.service_area_id=service_area.id
  where is_order_fake=0 and orders.status=9;""")
  rows2=cursors2.fetchall()
  data2=pd.DataFrame(rows2)
  arra1=[]
  for col in range(0,len(data2.columns)):
    arra1.append(cursors2.description[col][0])
  arra1[5]='order_status'
  arra1[72]='area_id'
  data2.columns=arra1
  cursors2=connection.cursor()
  cursors2.execute("""
  select *,CONVERT_TZ(user.created_at, '+00:00', '+5:00') As created_at_p
  from user
  inner join address
  on user.id=address.user_id
  inner join service_area
  on address.service_area_id=service_area.id
  where user.role_id=3 and user.id not in (select customer_id from orders) and CONVERT_TZ(user.created_at, '+00:00', '+5:00')<  now() - interval 24 hour;
  """)
  rows2=cursors2.fetchall()
  user_never_order=pd.DataFrame(rows2)
  arra1=[]
  for col in range(0,len(user_never_order.columns)):
    arra1.append(cursors2.description[col][0])
  user_never_order.columns=arra1
  user_never_order['Time']=datetime.now()-user_never_order['created_at_p']

  user_never_order['percential']=user_never_order['Time'].rank(pct=True,ascending=False)
  user_never_order.sort_values('percential',inplace=True,ascending=False)
  complete=data2[data2['order_status']==9]
  complete.dropna(axis=0,subset=['accepted_at'],inplace=True)
  max_date = max(complete['accepted_at_P'])
  complete['Time']=max_date-complete['accepted_at_P']
  User_recency=complete.loc[complete.groupby("customer_id")["Time"].idxmin()]
  fisrt_thirty_data=User_recency[User_recency['Time']<='30 days']
  fisrt_thirty_data['Percential']=fisrt_thirty_data['Time'].rank(pct=True)
  fisrt_thirty_data['rececy_Percential']=fisrt_thirty_data['Time'].rank(pct=True)
  #Frequency of all user:
  complete2=data2[data2['order_status']==9]
  complete2.dropna(axis=0,subset=['accepted_at'],inplace=True)
  complete2['freq'] = complete2.groupby('customer_id')['total'].transform('count')
  frquency_data=complete2
  frquency_data.drop_duplicates(subset ="customer_id",keep = 'first', inplace = True)
  frquency_data.sort_values('freq',inplace=True)
  recency_frequecy_data = pd.merge(left=fisrt_thirty_data, right=frquency_data, how='inner', left_on='customer_id', right_on='customer_id')
  recency_frequecy_data['Percential_freq']=recency_frequecy_data['freq'].rank(pct=True,ascending=False)
  recency_frequecy_data['reatation Time']=recency_frequecy_data['Time']
  recency_frequecy_data['reatation Percential']=recency_frequecy_data['Percential']
  recency_frequecy_data['freq User']=recency_frequecy_data['freq']
  recency_frequecy_data['frequency percential']=recency_frequecy_data['Percential_freq']
  recency_frequecy_data['Final Percential']=recency_frequecy_data['reatation Percential']*0.8+recency_frequecy_data['frequency percential']*0.20
  recency_frequecy_data.sort_values('Final Percential',inplace=True,ascending=False)
  above_thirty=User_recency[User_recency['Time']>'30 days']
  above_thirty['Percential']=above_thirty['Time'].rank(pct=True,ascending=False)
  reactivation_frequency = pd.merge(left=above_thirty, right=frquency_data, how='inner', left_on='customer_id', right_on='customer_id')
  reactivation_frequency['Percential_freq']=reactivation_frequency['freq'].rank(pct=True,ascending=True)
  reactivation_frequency['reatation Time']=reactivation_frequency['Time']
  reactivation_frequency['reatation Percential']=reactivation_frequency['Percential']
  reactivation_frequency['freq User']=reactivation_frequency['freq']
  reactivation_frequency['frequency percential']=reactivation_frequency['Percential_freq']
  reactivation_frequency['Final Percential']=reactivation_frequency['reatation Percential']*0.50+reactivation_frequency['frequency percential']*0.50
  reactivation_frequency.sort_values('Final Percential',inplace=True,ascending=False)
  user_never_order.columns=['customer_id', 'user_uuid', 'name', 'email', 'phone', 'role_id', 'locale_id','created_at', 'updated_at', 'password', 'is_active', 'avatar','phone_verified', 'device_token', 'sign_up_type', 'is_password_set','balance', 'organization_id', 'signup_discount_count', 'device_type','device_id', 'social_account_id', 'is_email_invitation_received','is_email_verified', 'inactive_comment', 'inactive_reason_id', 'id','user_id', 'location_name', 'poi', 'address', 'city_id', 'latitude','longitude', 'status', 'additional_details', 'service_area_id','created_at', 'updated_at', 'updated_by', 'created_by', 'id', 'name','city_id', 'geofence', 'is_active', 'dispatch_algo_id','dispatch_radius', 'sort_type', 'operational_time_msg', 'created_at','updated_at', 'created_by', 'updated_by', 'area_id', 'created_at_p','Time', 'percential']
  retained_user_cohort=0.45
  reactivated_user_cohort=0.45
  new_user_cohort=0.10
  Retention_reactivation={}
  new_user={}
  i=0
  recency_frequecy_data_log={}
  reactivation_frequency_log={}
  df=pd.DataFrame()
  for x in Area['Zone']:
    inverUserCount=int(float(Area[Area['Zone']==x]['Invite'])*retained_user_cohort)
    a=pd.DataFrame(recency_frequecy_data[(recency_frequecy_data['service_area_id_y']==x)].head(inverUserCount))
    inverUserCount=int(float(Area[Area['Zone']==x]['Invite'])*reactivated_user_cohort)
    b=pd.DataFrame(reactivation_frequency[(reactivation_frequency['service_area_id_y']==x)].head(inverUserCount))
    inverUserCount=int(float(Area[Area['Zone']==x]['Invite'])*new_user_cohort)
    c=pd.DataFrame(user_never_order[(user_never_order['service_area_id']==x)].head(inverUserCount))
    recency_frequecy_data_log[i]=pd.DataFrame(a)
    reactivation_frequency_log[i]=pd.DataFrame(b)


    new_user[i]=pd.DataFrame(c)
    i=i+1
  retentionData=pd.DataFrame()
  reactivationData=pd.DataFrame()
  NewUserData=pd.DataFrame()

  for x in range(0,i):
    #print(x)
    df2=pd.DataFrame(recency_frequecy_data_log[x]['customer_id'])
    df3=pd.DataFrame(reactivation_frequency_log[x]['customer_id'])
    df1=pd.DataFrame(new_user[x]['customer_id'])
    #df1.rename(columns={'id': 'customer_id'})
    retentionData=retentionData.append(df2)
    reactivationData=reactivationData.append(df3)
    NewUserData=NewUserData.append(df1)
    df=df.append(df1)
    df=df.append(df2)
    df=df.append(df3)

  df=df.reset_index(drop=True)


In [ ]:
df

NameError: ignored

In [ ]:
retentionData=pd.DataFrame()
reactivationData=pd.DataFrame()
NewUserData=pd.DataFrame()

retentionData.to_csv('retentionData.csv')
!cp retentionData.csv "drive/My Drive/"
reactivationData.to_csv('reactivationData.csv')
!cp reactivationData.csv "drive/My Drive/"
NewUserData.to_csv('NewUserData.csv')
!cp NewUserData.csv "drive/My Drive/"

In [ ]:
reactivation_frequency_log[1].shape

NameError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
retentionData.shape

(0, 0)

In [ ]:
df.shape

NameError: ignored